In [ ]:
from numba import cuda 
device = cuda.get_current_device()
device.reset()

In [1]:
import pandas as pd
from keras.callbacks import History, ReduceLROnPlateau,EarlyStopping,ModelCheckpoint
import os
import numpy as np
from data_analysis import calculate_metrics
from functools import partial
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import pickle
import dill
from hyper_mining import objective_fn

Using TensorFlow backend.
C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\tomas\miniconda3\envs\binding

In [2]:
fspace = {
    'conv1' : hp.quniform('conv1', 32, 64, 8),
    'conv2' : hp.quniform('conv2', 64, 128, 8),
    'conv3' : hp.quniform('conv3', 128, 168, 8),
    'fp' : hp.quniform('fp', 96, 196, 8),
    'dense1' : hp.quniform('dense1',96,512,32),
    'dense2' : hp.quniform('dense2',96,512,32),
    'dense3' : hp.quniform('dense3',64,512,32),
    'dropout_rate' : hp.uniform('dropout_rate',0.1,0.5),
    'lr' : hp.uniform('lr',0.000001,0.01),
    'n_epochs' : hp.quniform('n_epochs',15,60,5),
    'batch_size' : hp.quniform('batch_size',64,256,16),
    'colsample_bylevel' : hp.uniform('colsample_bylevel', 0.1, 1), 
    'colsample_bytree' : hp.uniform('colsample_bytree', 0.1, 1), 
    'gamma' : hp.uniform('gamma', 0.1, 1), 
    'learning_rate' : hp.uniform('learning_rate', 0.1, 1),
    'max_delta_step' : hp.quniform('max_delta_step',1,10,1),
    'max_depth' : hp.quniform('max_depth',6, 12, 1),
    'min_child_weight' : hp.quniform('min_child_weight',10 ,500 ,5),
    'reg_alpha' : hp.uniform('reg_alpha',0.1,100),
    'reg_lambda' : hp.uniform('reg_lambda',0.1,100),
    'subsample' : hp.uniform('subsample',0.1,1.0),
    'max_bin' : hp.quniform('max_bin',16,256,16)
    #'margin' : hp.uniform('margin',0.2,2)
}

In [3]:
target = 'p38'
base_path = f'C:/Users/tomas/Documents/GitHub/kinase_binding'

data_fpath = base_path+f'/data/{target}/data.csv'
df=pd.read_csv(data_fpath).set_index('biolab_index')

with open(base_path+f'/data/{target}/train_val_folds.pkl', "rb") as in_f:
    train_val_folds = dill.load(in_f)
with open(base_path+f'/data/{target}/train_test_folds.pkl', "rb") as in_f:
    train_test_folds = dill.load(in_f)

In [8]:
train = df.loc[train_test_folds[0]]
val = df.loc[train_test_folds[1]]

In [9]:
val

,rdkit,standard_value,Binary,assay_type,not_sure
6694856396823213521,CSc1nc(-c2ccc(F)cc2)c(-c2ccnc(NC(=O)c3ccccc3)c...,6.472370,0,binding,1
-2189404736969118545,ClC1=C(Cl)[C@]2(Cl)[C@@H]3[C@@H]4C[C@H]([C@@H]...,4.364587,0,binding,0
-4240361026298800667,CCn1c(SC)nc(-c2ccc(F)cc2)c1-c1ccnc(NC(=O)/C=C/...,7.420216,1,binding,1
1704799582349985732,C[N+](C)(C)CCO.Cn1c(-c2cn(-c3cccc(C(=O)[O-])c3...,7.040959,1,binding,1
1624810325886590497,Cn1c(-c2cn(-c3cccc(C(=O)NCCCO)c3)nn2)nc(-c2ccc...,6.537602,0,binding,1
...,...,...,...,...,...
8379113105631449385,COc1ccccc1-c1c(N)c(C(=O)c2ccc(F)cc2F)cc[n+]1[O-],7.638272,1,binding,0
-4924377680869854252,Cn1nc(C(C)(C)C)cc1NC(=O)Nc1ccc(Cc2ccc(NC(=O)OC...,7.060481,1,binding,1
7108923260425567357,Cn1c(=O)c(Cc2ccc(F)cc2F)cc2cnnc(-c3ccc(F)cc3F)c21,8.214670,1,binding,0
347631336707966323,OCCNc1cc(-c2[nH]c3cccnc3c2-c2ccc(F)cc2)ccn1,8.522879,1,binding,0


In [4]:
training_list = [df.loc[train_val_folds[0][0]],
                 df.loc[train_val_folds[1][0]],
                 df.loc[train_val_folds[2][0]],
                 df.loc[train_val_folds[3][0]],
                 df.loc[train_val_folds[4][0]],
                 df.loc[train_val_folds[5][0]],
                 ]
validation_list = [df.loc[train_val_folds[0][1]],
                   df.loc[train_val_folds[1][1]],
                   df.loc[train_val_folds[2][1]],
                   df.loc[train_val_folds[3][1]],
                   df.loc[train_val_folds[4][1]],
                   df.loc[train_val_folds[5][1]],
                   ]

In [5]:
fmin_objective = partial(objective_fn, train_sets = training_list, val_sets = validation_list)

In [6]:
def run_trials():

    trials_step = 0  # how many additional trials to do after loading saved trials. 1 = save after iteration
    max_trials = 1  # initial max_trials. put something small to not have to wait

    
    try:  # try to load an already saved trials object, and increase the max
        trials = pickle.load(open("gcn_xgb.hyperopt", "rb"))
        print("Found saved Trials! Loading...")
        max_trials = len(trials.trials) + trials_step
        print("Rerunning from {} trials to {} (+{}) trials".format(len(trials.trials), max_trials, trials_step))
    except:  # create a new trials object and start searching
        trials = Trials()

    best = fmin(fn = fmin_objective, space = fspace, algo=tpe.suggest, max_evals=max_trials, trials=trials)

    print("Best:", best)
    
    # save the trials object
    with open("gcn_xgb.hyperopt", "wb") as f:
        pickle.dump(trials, f)
    return(trials)

In [7]:
trials = run_trials()

Found saved Trials! Loading...
Rerunning from 154 trials to 154 (+0) trials
100%|████████████████████████████████████████████████████████████████████████| 154/154 [00:00<?, ?trial/s, best loss=?]
Best: {'batch_size': 288.0, 'colsample_bylevel': 0.4371082812232264, 'colsample_bytree': 0.4179415558635843, 'conv1': 56.0, 'conv2': 88.0, 'conv3': 136.0, 'dense1': 384.0, 'dense2': 288.0, 'dense3': 224.0, 'dropout_rate': 0.27225175676555935, 'fp': 152.0, 'gamma': 0.919836526180396, 'learning_rate': 0.41409388868400826, 'lr': 0.0008110012706176706, 'max_bin': 48.0, 'max_delta_step': 2.0, 'max_depth': 7.0, 'min_child_weight': 20.0, 'n_epochs': 25.0, 'reg_alpha': 42.8887552483495, 'reg_lambda': 12.306130216692438, 'subsample': 0.6038298323514097}


In [14]:
best_loss = trials.trials[0]['result']['loss']
for i in range(1,len(trials.trials)):
    if (trials.trials[i]['result']['loss'] <=  best_loss):
        best_loss = trials.trials[i]['result']['loss']
        index = i
best_params = trials.trials[index]['misc']['vals']

In [9]:
best_params

{'batch_size': [288.0],
 'colsample_bylevel': [0.4371082812232264],
 'colsample_bytree': [0.4179415558635843],
 'conv1': [56.0],
 'conv2': [88.0],
 'conv3': [136.0],
 'dense1': [384.0],
 'dense2': [288.0],
 'dense3': [224.0],
 'dropout_rate': [0.27225175676555935],
 'fp': [152.0],
 'gamma': [0.919836526180396],
 'learning_rate': [0.41409388868400826],
 'lr': [0.0008110012706176706],
 'max_bin': [48.0],
 'max_delta_step': [2.0],
 'max_depth': [7.0],
 'min_child_weight': [20.0],
 'n_epochs': [25.0],
 'reg_alpha': [42.8887552483495],
 'reg_lambda': [12.306130216692438],
 'subsample': [0.6038298323514097]}

In [15]:
from hyper_mining import XGB_predictor,GCN_online_mining_test
from data_analysis import calculate_metrics
es = EarlyStopping(monitor='loss',patience=8, min_delta=0)
rlr = ReduceLROnPlateau(monitor='loss',factor=0.5, patience=4, verbose=1, min_lr=0.0000001)
gcn_best = {
        "num_layers" : 3,
        "max_atoms" : 70,
        "num_atom_features" : 62,
        "num_atom_features_original" : 62,
        "num_bond_features" : 6,
        "max_degree" : 5,
        "conv_width" : [int(best_params['conv1'][0]), int(best_params['conv2'][0]), int(best_params['conv3'][0])],
        "fp_length" : [int(best_params['fp'][0]), int(best_params['fp'][0]), int(best_params['fp'][0])],
        "activ_enc" : "selu",
        "activ_dec" : "selu",
        "learning_rates" : [0.001,0.001,0.001],
        "learning_rates_fp": [0.005,0.005,0.005],
        "losses_conv" : {
                    "neighbor_output": "mean_squared_error",
                    "self_output": "mean_squared_error",
                    },
        "lossWeights" : {"neighbor_output": 1.0, "self_output": 1.0},
        "metrics" : "mse",
        "loss_fp" : "mean_squared_error",
        "enc_layer_names" : ["enc_1", "enc_2", "enc_3"],
        'callbacks' : [es,rlr],
        'adam_decay': 0.0005329142291371636,
        'beta': 5,
        'p': 0.004465204118126482,
        'dense_size' : [int(best_params['dense1'][0]), int(best_params['dense2'][0]), int(best_params['dense3'][0])],
        'dropout_rate' : [best_params['dropout_rate'][0], best_params['dropout_rate'][0]],
        'lr' : best_params['lr'][0],
        'batch_size' : int(best_params['batch_size'][0]),
        'n_epochs' : int(best_params['n_epochs'][0])
        #'margin' : best_params['margin'][0]
        }
xgb_best = {
        "colsample_bylevel" : best_params['colsample_bylevel'][0],
        "colsample_bytree" : best_params['colsample_bytree'][0],
        "gamma" : best_params['gamma'][0],
        "eta" : best_params['learning_rate'][0],
        "max_delta_step" : int(best_params['max_delta_step'][0]),
        "max_depth" : int(best_params['max_depth'][0]),
        "min_child_weight" : int(best_params['min_child_weight'][0]),
        "alpha" : best_params['reg_alpha'][0],
        "lambda" : best_params['reg_lambda'][0],
        "subsample" : best_params['subsample'][0],
        "max_bin" : int(best_params['max_bin'][0]),
        "eval_metric":'auc',
        "objective":'binary:logistic',
        "booster":'gbtree'
        #"single_precision_histogram" : True
        }
class_XGB = XGB_predictor(xgb_best)
class_GCN = GCN_online_mining_test(gcn_best)

In [ ]:
xgb_hyper = {
        "colsample_bylevel" : 0.5612301667238877,
        "colsample_bytree" : 0.788688363076523,
        "gamma" : 0.35376030016117566,
        "eta" : 0.4023692255888918,
        "max_delta_step" : int(3),
        "max_depth" : int(8),
        "min_child_weight" : int(70),
        "alpha" : 0.15030685758880047,
        "lambda" : 15.311721955443915,
        "subsample" : 0.8303923929525608,
        "max_bin" : int(208),
        "eval_metric":'auc',
        "objective":'binary:logistic',
        "booster":'gbtree',
        "single_precision_histogram" : True
}
class_XGB_2 = XGB_predictor(xgb_hyper)

In [11]:
#K.clear_session()
training_metrics = {}
validation_metrics = {}
es2 = EarlyStopping(monitor='loss',patience=15, min_delta=0)
rlr2 = ReduceLROnPlateau(monitor='loss',factor=0.5, patience=2, verbose=1, min_lr=0.000000001)
for i in range(len(training_list)):
        X_atoms_cold,X_bonds_cold,X_edges_cold = class_GCN.dataframe_to_gcn_input(validation_list[i])
        Y_cold = validation_list[i].Binary 
        Y_dummy_cold = np.empty((X_atoms_cold.shape[0],gcn_best['dense_size'][2]+1))
        X_atoms_train, X_bonds_train, X_edges_train = class_GCN.dataframe_to_gcn_input(training_list[i])
        Y = training_list[i].Binary
        Y_dummy_train = np.empty((X_atoms_train.shape[0],gcn_best['dense_size'][2]+1))
        
        gcn_encoder = class_GCN.build_encoder()
        gcn_model = class_GCN.build_model(gcn_encoder)
        gcn_mining = class_GCN.build_mining(gcn_model)
        
        gcn_mining.fit([X_atoms_train,X_bonds_train,X_edges_train,Y],
                       Y_dummy_train,
                       epochs = gcn_best['n_epochs'],
                       batch_size = gcn_best['batch_size'],
                       shuffle = True,
                       validation_data = ([X_atoms_cold,X_bonds_cold,X_edges_cold,Y_cold],Y_dummy_cold),
                       callbacks=[es2,rlr2]
                      )
        #Predict Embeddings
        embeddings_cold = gcn_model.predict([X_atoms_cold,X_bonds_cold,X_edges_cold])
        embeddings_train = gcn_model.predict([X_atoms_train, X_bonds_train, X_edges_train])
        
        #Prepare data for XGBoost
        dmatrix_train = class_XGB.to_xgb_input(Y,embeddings_train)
        dmatrix_cold = class_XGB.to_xgb_input(Y_cold,embeddings_cold)
        
        evalist = [(dmatrix_train,'train'),(dmatrix_cold,'eval')]
        xgb_model = class_XGB.build_model(dmatrix_train,evalist,300)
        
        xgb_pred_cold = xgb_model.predict(dmatrix_cold)
        validation_metrics['Val_%s'%i] = calculate_metrics(np.array(Y_cold),xgb_pred_cold)
        
        xgb_pred_train = xgb_model.predict(dmatrix_train)
        training_metrics['Train_%s'%i] = calculate_metrics(np.array(Y),xgb_pred_train)
        


LAYER 0
LAYER 1
LAYER 2


C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 2541 samples, validate on 509 samples
Epoch 1/25
2541/2541 [==============================] - 7s 3ms/step - loss: 0.9973 - val_loss: 0.9967
Epoch 2/25
2541/2541 [==============================] - 1s 265us/step - loss: 0.9967 - val_loss: 0.9881
Epoch 3/25
2541/2541 [==============================] - 1s 265us/step - loss: 0.9941 - val_loss: 0.9804
Epoch 4/25
2541/2541 [==============================] - 1s 266us/step - loss: 0.9897 - val_loss: 0.9772
Epoch 5/25
2541/2541 [==============================] - 1s 268us/step - loss: 0.9861 - val_loss: 0.9742
Epoch 6/25
2541/2541 [==============================] - 1s 264us/step - loss: 0.9849 - val_loss: 0.9741
Epoch 7/25
2541/2541 [==============================] - 1s 264us/step - loss: 0.9815 - val_loss: 0.9704
Epoch 8/25
2541/2541 [==============================] - 1s 262us/step - loss: 0.9760 - val_loss: 0.9727
Epoch 9/25
2541/2541 [==============================] - 1s 262us/step - loss: 0.9767 - val_loss: 0.9698
Epoch 10/25
2541/25

C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 2541 samples, validate on 509 samples
Epoch 1/25
2541/2541 [==============================] - 6s 2ms/step - loss: 0.9974 - val_loss: 0.9980
Epoch 2/25
2541/2541 [==============================] - 1s 266us/step - loss: 0.9966 - val_loss: 0.9970
Epoch 3/25
2541/2541 [==============================] - 1s 265us/step - loss: 0.9954 - val_loss: 0.9871
Epoch 4/25
2541/2541 [==============================] - 1s 265us/step - loss: 0.9910 - val_loss: 0.9790
Epoch 5/25
2541/2541 [==============================] - 1s 266us/step - loss: 0.9873 - val_loss: 0.9822
Epoch 6/25
2541/2541 [==============================] - 1s 265us/step - loss: 0.9851 - val_loss: 0.9749
Epoch 7/25
2541/2541 [==============================] - ETA: 0s - loss: 0.983 - 1s 264us/step - loss: 0.9822 - val_loss: 0.9753
Epoch 8/25
2541/2541 [==============================] - 1s 264us/step - loss: 0.9806 - val_loss: 0.9809
Epoch 9/25
2541/2541 [==============================] - 1s 265us/step - loss: 0.9787 - val_loss: 0.

...

Train on 2541 samples, validate on 509 samples
Epoch 1/25
2541/2541 [==============================] - 8s 3ms/step - loss: 0.9972 - val_loss: 0.9982
Epoch 2/25
2541/2541 [==============================] - 1s 271us/step - loss: 0.9965 - val_loss: 0.9959
Epoch 3/25
2541/2541 [==============================] - 1s 273us/step - loss: 0.9949 - val_loss: 0.9801
Epoch 4/25
2541/2541 [==============================] - 1s 269us/step - loss: 0.9895 - val_loss: 0.9831
Epoch 5/25
2541/2541 [==============================] - 1s 269us/step - loss: 0.9858 - val_loss: 0.9814
Epoch 6/25
2541/2541 [==============================] - 1s 270us/step - loss: 0.9831 - val_loss: 0.9772
Epoch 7/25
2541/2541 [==============================] - 1s 269us/step - loss: 0.9793 - val_loss: 0.9770
Epoch 8/25
2541/2541 [==============================] - 1s 269us/step - loss: 0.9781 - val_loss: 0.9596
Epoch 9/25
2541/2541 [==============================] - 1s 270us/step - loss: 0.9740 - val_loss: 0.9759
Epoch 10/25
2541/25

C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 2541 samples, validate on 509 samples
Epoch 1/25
2541/2541 [==============================] - 9s 3ms/step - loss: 0.9975 - val_loss: 0.9984
Epoch 2/25
2541/2541 [==============================] - 1s 269us/step - loss: 0.9969 - val_loss: 0.9922
Epoch 3/25
2541/2541 [==============================] - 1s 279us/step - loss: 0.9945 - val_loss: 0.9819
Epoch 4/25
2541/2541 [==============================] - ETA: 0s - loss: 0.989 - 1s 270us/step - loss: 0.9889 - val_loss: 0.9808
Epoch 5/25
2541/2541 [==============================] - 1s 271us/step - loss: 0.9862 - val_loss: 0.9739
Epoch 6/25
2541/2541 [==============================] - 1s 275us/step - loss: 0.9830 - val_loss: 0.9788
Epoch 7/25
2541/2541 [==============================] - 1s 272us/step - loss: 0.9823 - val_loss: 0.9475
Epoch 8/25
2541/2541 [==============================] - 1s 271us/step - loss: 0.9769 - val_loss: 0.9792
Epoch 9/25
2541/2541 [==============================] - 1s 270us/step - loss: 0.9805 - val_loss: 0.

C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 2545 samples, validate on 505 samples
Epoch 1/25
2545/2545 [==============================] - 10s 4ms/step - loss: 0.9976 - val_loss: 0.9976
Epoch 2/25
2545/2545 [==============================] - 1s 270us/step - loss: 0.9971 - val_loss: 0.9961
Epoch 3/25
2545/2545 [==============================] - 1s 273us/step - loss: 0.9950 - val_loss: 0.9835
Epoch 4/25
2545/2545 [==============================] - 1s 270us/step - loss: 0.9904 - val_loss: 0.9776
Epoch 5/25
2545/2545 [==============================] - 1s 271us/step - loss: 0.9862 - val_loss: 0.9769
Epoch 6/25
2545/2545 [==============================] - 1s 270us/step - loss: 0.9836 - val_loss: 0.9777
Epoch 7/25
2545/2545 [==============================] - 1s 270us/step - loss: 0.9808 - val_loss: 0.9805
Epoch 8/25
2545/2545 [==============================] - 1s 272us/step - loss: 0.9774 - val_loss: 0.9774
Epoch 9/25
2545/2545 [==============================] - 1s 283us/step - loss: 0.9774 - val_loss: 0.9777
Epoch 10/25
2545/2

In [17]:
es2 = EarlyStopping(monitor='loss',patience=15, min_delta=0)
rlr2 = ReduceLROnPlateau(monitor='loss',factor=0.5, patience=2, verbose=1, min_lr=0.000000001)
X_atoms_cold,X_bonds_cold,X_edges_cold = class_GCN.dataframe_to_gcn_input(val)
Y_cold = val.Binary 
Y_dummy_cold = np.empty((X_atoms_cold.shape[0],gcn_best['dense_size'][2]+1))
X_atoms_train, X_bonds_train, X_edges_train = class_GCN.dataframe_to_gcn_input(train)
Y = train.Binary
Y_dummy_train = np.empty((X_atoms_train.shape[0],gcn_best['dense_size'][2]+1))
        
gcn_encoder = class_GCN.build_encoder()
gcn_model = class_GCN.build_model(gcn_encoder)
gcn_mining = class_GCN.build_mining(gcn_model)
gcn_mining.fit([X_atoms_train,X_bonds_train,X_edges_train,Y],
                       Y_dummy_train,
                       epochs = gcn_best['n_epochs'],
                       batch_size = gcn_best['batch_size'],
                       shuffle = True,
                       validation_data = ([X_atoms_cold,X_bonds_cold,X_edges_cold,Y_cold],Y_dummy_cold),
                       callbacks=[es2,rlr2]
                      )
        #Predict Embeddings
embeddings_cold = gcn_model.predict([X_atoms_cold,X_bonds_cold,X_edges_cold])
embeddings_train = gcn_model.predict([X_atoms_train, X_bonds_train, X_edges_train])
        
        #Prepare data for XGBoost
dmatrix_train = class_XGB.to_xgb_input(Y,embeddings_train)
dmatrix_cold = class_XGB.to_xgb_input(Y_cold,embeddings_cold)
        
evalist = [(dmatrix_train,'train'),(dmatrix_cold,'eval')]
xgb_model = class_XGB.build_model(dmatrix_train,evalist,300)
        
xgb_pred_cold = xgb_model.predict(dmatrix_cold)
validation_metrics = calculate_metrics(np.array(Y_cold),xgb_pred_cold)
        
xgb_pred_train = xgb_model.predict(dmatrix_train)
training_metrics = calculate_metrics(np.array(Y),xgb_pred_train)

LAYER 0
LAYER 1
LAYER 2


C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 3050 samples, validate on 509 samples
Epoch 1/25
3050/3050 [==============================] - 9s 3ms/step - loss: 0.9971 - val_loss: 0.9970
Epoch 2/25
3050/3050 [==============================] - 1s 266us/step - loss: 0.9957 - val_loss: 0.9881
Epoch 3/25
3050/3050 [==============================] - 1s 267us/step - loss: 0.9912 - val_loss: 0.9847
Epoch 4/25
3050/3050 [==============================] - 1s 267us/step - loss: 0.9873 - val_loss: 0.9864
Epoch 5/25
3050/3050 [==============================] - 1s 270us/step - loss: 0.9838 - val_loss: 0.9846
Epoch 6/25
3050/3050 [==============================] - 1s 266us/step - loss: 0.9811 - val_loss: 0.9841
Epoch 7/25
3050/3050 [==============================] - 1s 270us/step - loss: 0.9790 - val_loss: 0.9835
Epoch 8/25
3050/3050 [==============================] - 1s 273us/step - loss: 0.9761 - val_loss: 0.9834
Epoch 9/25
3050/3050 [==============================] - 1s 267us/step - loss: 0.9761 - val_loss: 0.9816
Epoch 10/25
3050/30

In [18]:
validation_metrics

{'roc_auc': 0.8126307994462152,
 'tn': 235,
 'fp': 71,
 'fn': 63,
 'tp': 140,
 'map': 0.7295864861386149,
 'precision': 0.6635071090047393,
 'recall': 0.6896551724137931,
 'accuracy': 0.7367387033398821}

In [19]:
training_metrics

{'roc_auc': 0.9511664222545811,
 'tn': 1335,
 'fp': 231,
 'fn': 116,
 'tp': 1368,
 'map': 0.939258856153558,
 'precision': 0.8555347091932458,
 'recall': 0.921832884097035,
 'accuracy': 0.8862295081967213}